1. обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
2. при обучении моделей обязательно использовать кроссвалидацию
3. вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)
4. сделать выводы о том, какая модель справилась с задачей лучше других
5. (опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого). 

In [1]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import precision_recall_curve

In [2]:
df = pd.read_csv('train_case2.csv', ';')
df.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


Объективные признаки:

 - Возраст
 - Рост
 - Вес
 - Пол
 

Результаты измерения:

 - Артериальное давление верхнее и нижнее
 - Холестерин
 - Глюкоза
 

Субъективные признаки:

 - Курение
 - Употребление Алкоголя
 - Физическая активность

Создадим классы для выбора и обработки данных

In [3]:
# Трансформатор для выбора одного столбца из фрейма данных для выполнения дополнительных преобразований 
class ColumnSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
# Выбор значений числового признака
class NumberSelector(BaseEstimator, TransformerMixin):

    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
# Dummies кодирование
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio',1), df['cardio'], test_size = 0.3, random_state = 42)

Создадим списки признаков для разных пайплайнов и списки, в которые будем складывать пайплайны

In [5]:
continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

Создадим пайплайны для каждого признака по выше созданым категориям

In [6]:
for cont_col in continuos_cols:
    # создаем объект класса Pipeline
    transfomer =  Pipeline([
        # вызываем класс выбора признака для значений признака cont_col
                ('selector', NumberSelector(key=cont_col)),
        # применяем стандартизацию для выбранного признака
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    


    
for cat_col in cat_cols:
    # создаем объект класса Pipeline
    cat_transformer = Pipeline([
        # вызываем класс выбора признаков из cat_col
                ('selector', ColumnSelector(key=cat_col)),
        # преобразуем их в dummies переменные
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
    
for base_col in base_cols:
    # создаем объекст класса Pipeline
    base_transformer = Pipeline([
        ('selector', NumberSelector(key = base_col)),
    ])
    base_transformers.append((base_col, base_transformer))

Объединим все трансформеры в один и передадим список полученных трансформеров

In [7]:
feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)

feature_proc = Pipeline([('feats', feats)])

Теперь создадим 3 пайплайна, в которые передадим 3 разных классификатора(3 модели)

In [8]:
class_1 = Pipeline([('feats', feats),
                   ('class_1', LogisticRegression())])

class_2 = Pipeline([('feats', feats),
                   ('class_2', RandomForestClassifier())])

class_3 = Pipeline([('feats', feats),
                   ('class_3', CatBoostClassifier(silent = True))])

Запустим для каждого кросс-валидацию с разделением на 10 фолдов

In [9]:
crosval_1 = cross_val_score(class_1, X_train, y_train, cv = 10, scoring = 'roc_auc')
crosval_2 = cross_val_score(class_2, X_train, y_train, cv = 10, scoring = 'roc_auc')
crosval_3 = cross_val_score(class_3, X_train, y_train, cv = 10, scoring = 'roc_auc')

print(f'CrossValScore LogReg {np.mean(crosval_1)} c std {np.std(crosval_1)}')
print(f'CrossValScore RFC {np.mean(crosval_2)} c std {np.std(crosval_2)}')
print(f'CrossValScore CatBoost {np.mean(crosval_3)} c std {np.std(crosval_3)}')

CrossValScore LogReg 0.7801199973877395 c std 0.0057060030775680235
CrossValScore RFC 0.7738749620155989 c std 0.005826362129858266
CrossValScore CatBoost 0.801085804096451 c std 0.005188089205718202


Обучим классификаторы

In [10]:
class_1.fit(X_train, y_train)
class_2.fit(X_train, y_train)
class_3.fit(X_train, y_train)

Pipeline(steps=[('feats',
                 FeatureUnion(transformer_list=[('age',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='age')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('height',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='height')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('weight',
                                                 Pipeline(steps=[('selector',
                                           

Найдем интересующие нас метрики по каждому классификатору и выведем их в виде DataFrame

In [11]:
precision_1, recall_1, threshold_1 = precision_recall_curve(y_test.values, class_1.predict_proba(X_test)[:,1])
precision_2, recall_2, threshold_2 = precision_recall_curve(y_test.values, class_2.predict_proba(X_test)[:,1])
precision_3, recall_3, threshold_3 = precision_recall_curve(y_test.values, class_3.predict_proba(X_test)[:,1])

#Найдем F-меру для каждого классификатора
b = 1
f_score_1 = (1 + b**2)*(precision_1 * recall_1)/(b**2*precision_1 + recall_1)
f_score_2 = (1 + b**2)*(precision_2 * recall_2)/(b**2*precision_2 + recall_2)
f_score_3 = (1 + b**2)*(precision_3 * recall_3)/(b**2*precision_3 + recall_3)

# найдем индекс максимального значения F-меры. При этом эти же индексы будут принадлежать оптимальным значениям 
# точности, полноты и порога
ix1 = np.argmax(f_score_1)
ix2 = np.argmax(f_score_2)
ix3 = np.argmax(f_score_3)

# выведем все значения
print(f'Для LogReg: порог {round(threshold_1[ix1],2)}, '
      f'f_score {round(f_score_1[ix1],2)}, точность {round(precision_1[ix1],2)}, полнота {round(recall_1[ix1],2)}')
print(f'Для RFC: порог {round(threshold_2[ix2],2)}, '
      f'f_score {round(f_score_2[ix2],2)}, точность {round(precision_2[ix2],2)} полнота {round(recall_2[ix2],2)}')
print(f'Для CatBoost: порог {round(threshold_3[ix3],2)}, '
      f'f_score {round(f_score_3[ix3],2)}, точность {round(precision_3[ix3],2)} полнота {round(recall_3[ix3],2)}')

Для LogReg: порог 0.41, f_score 0.73, точность 0.66, полнота 0.82
Для RFC: порог 0.44, f_score 0.72, точность 0.69 полнота 0.76
Для CatBoost: порог 0.39, f_score 0.75, точность 0.7 полнота 0.8


In [12]:
pd.DataFrame(data = [[f_score_1[ix1],precision_1[ix1],recall_1[ix1],threshold_1[ix1]],
                    [f_score_2[ix2], precision_2[ix2],recall_2[ix2],threshold_2[ix2]],
                    [f_score_3[ix3],precision_3[ix3],recall_3[ix3],threshold_3[ix3]]],
            index = ['LogReg','RFC', 'CatBoost'],
            columns = ['F_score','precision','recall','threshold'])

,F_score,precision,recall,threshold
LogReg,0.734388,0.662671,0.823513,0.410103
RFC,0.722441,0.689798,0.758326,0.440000
CatBoost,0.746034,0.698180,0.800930,0.392537


Лучше всего справился CatBoost, что не удивительно, но сама модель довольно сложна. 

LogRed не сильно уступает(спорный вопрос про точность), я бы выбрал LogRed из-за простоты алгоритма.

### Какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов?

Очень часто в задачах мы имеем дизбаланс в сторону класса 0, поэтому в оценке работы модели нам нужно учитывать данные особенности(балансировка не всегда хорошее решение).

В AUC-ROC FPR считается отсносительно общего числа отрицательных объектов, если у нас будет большой дизбаланс в сторону класса 0, то AUC-ROC посчитает классификатор чуть-ли не идеальным, когда на самом деле он бесполезен.

В таком случае следует смотреть на AUC-PR, в ней по осям откладываются точность и полнота.